$$
\def\E{\mathbb{E}}
E_1 = \E[\max(0, A(T) - L(1+s))]
$$

$$
B_1 = \E[ \min(L(1+s), A(T)) ] \geq L
$$

$$
E_2 = \E[ \max(0, A(T) + C - L(1+s) - C(1+s')) ] \geq E_1
$$

$$
B_2 = \E[ \min(L(1+s), (A(T) + C) \frac{L}{L + C} ) ]
$$

$$
\begin{align}
B_2' &= \E[ \min(C(1+s'),  (A(T) + C) \frac{C}{L+C}) ] \geq C \\
&= \frac{C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C}, A(T) + C) ] \\
&= \frac{C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C} - C, A(T)) + C ] \\
&= \frac{C}{L+C} \E[ \min((1+s')(L+C) - C, A(T)) ] + \frac{C^2}{L+C} \\
\end{align}
$$

In [26]:
vol = 0.4

In [78]:
a0 = 89.9

In [79]:
L = 90.
C = 1.0

In [80]:
import numpy as np
import scipy as sp
import scipy.stats
import math

In [81]:
def bs(s, k):
    d1 = (math.log(s / k) + (0.5 * vol ** 2)) / vol
    d2 = (math.log(s / k) - (0.5 * vol ** 2)) / vol
    call_price = s * sp.stats.norm.cdf(d1) - k * sp.stats.norm.cdf(d2)   
    put_price = k * sp.stats.norm.cdf(-d2) - s * sp.stats.norm.cdf(-d1)
    return put_price, call_price

In [82]:
def b1(s):
    put, call = bs(a0, L * (1+s))
    return L * (1 + s) - put

In [83]:
b1(10)

89.899999981405017

In [84]:
def e1(s):
    put, call = bs(a0, L * (1+s))
    return call

In [85]:
e1(0)

14.208876069738949

In [86]:
res = sp.optimize.minimize(
    lambda s: (b1(s) - L) ** 2,
    [0.0],
    bounds=[(-0.1, 1000)]
)
res

      fun: array([ 0.01])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 8
      nit: 1
   status: 0
  success: True
        x: array([ 110.94636648])

In [87]:
s = res.x
s

array([ 110.94636648])

In [75]:
b1(s[0])

88.0

In [55]:
def b2prime(sprime):
    strike = (L + C) * (1 + sprime) - C
    put, call = bs(a0, strike)
    return C / (L + C) * (strike - put) + (C ** 2) / (L + C)

In [56]:
b2prime(0.1)

0.74178967426149933

In [57]:
res = sp.optimize.minimize(
    lambda s: (b2prime(s) - C) ** 2,
    [-0.15],
    bounds=[(-0.2, 1.5)]
)
res

      fun: array([ 0.04841792])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00037734])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 22
      nit: 10
   status: 0
  success: True
        x: array([ 1.5])

In [58]:
sprime = res.x
b2prime(res.x)

array([ 0.77995927])

In [59]:
def e2(sprime):
    put, call = bs(a0, L * (1 + s) + C * sprime)
    return call

In [60]:
e2(sprime)

array([  3.25356630e-33])

In [61]:
e1(s)

array([  3.26962683e-33])

In [22]:
sprime * C

array([ 0.47436557])

In [23]:
(s - sprime) * C

array([ 0.00535996])

In [24]:
e1(s) - e2(sprime)

array([ 0.09688755])

In [25]:
s


array([ 0.47972552])